In [1]:
from __future__ import division
import os
import time
from glob import glob
import tensorflow as tf
import numpy as np
from six.moves import xrange
import SharedArray as sa
from sklearn.utils import shuffle
from utils import *
from superGAN_ops import *

/Users/sbnahata/anaconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
class MidiNet(object):
    def __init__ (self, sess, 
                  batch_size=72, output_w=16, output_h=128,
                  gf_dim = 64, df_dim = 64, sample_size=72,
                  y_dim=13, c_dim=1, z_dim = 100):
        self.sess = sess
        self.is_grayscale = (c_dim == 1)
        self.batch_size = batch_size
        self.output_w = output_w
        self.output_h = output_h
        
        self.gf_dim = gf_dim
        self.df_dim = df_dim
        self.sample_size = sample_size
        
        self.y_dim = y_dim
        self.z_dim = z_dim
        self.c_dim = c_dim
        
        #batch normalization: deals with poor initialization and helps gradient flow
        self.d_bn0 = batch_norm(name='d_bn0')
        self.d_bn1 = batch_norm(name='d_bn1')
        self.d_bn2 = batch_norm(name='d_bn2')
        self.d_bn3 = batch_norm(name='d_bn3')
        
        self.g_prev_bn0 = batch_norm(name='g_prev_bn0')
        self.g_prev_bn1 = batch_norm(name='g_prev_bn1')
        self.g_prev_bn2 = batch_norm(name='g_prev_bn2')
        self.g_prev_bn3 = batch_norm(name='g_prev_bn3')
        
        self.g_bn0 = batch_norm(name='g_bn0')
        self.g_bn1 = batch_norm(name='g_bn1')
        self.g_bn2 = batch_norm(name='g_bn2')
        self.g_bn3 = batch_norm(name='g_bn3')
        self.g_bn4 = batch_norm(name='g_bn4')
        
        self.build_model()
        
    def build_model(self):
        self.y = tf.placeholder(tf.float32,[self.batch_size, self.y_dim], name='y')
        self.prev_bar = tf.placeholder(tf.float32, [self.batch_size] + [self.output_w, self.output_h, self.c_dim],
                                      name='prev_bar')
        self.images = tf.placeholder(tf.float32, [self.batch_size] + [self.output_w, self.output_h, self.c_dim],
                                    name='real_images')
        self.z = tf.placeholder(tf.float32, [None, self.z_dim], name='z')
        
        self.G = self.generator(self.z, self.y, self.prev_bar)
        self.D, self.D_logits, self.fm = self.discriminator(self.images, self.y, reuse=False)
        self.sampler = self.sampler(self.z, self.y, self.prev_bar)
        
        self.D_, self.D_logits_ , self.fm_ = self.discriminator(self.G, self.y, reuse=True)
    
        self.z_sum = tf.summary.histogram("z", self.z)
        
        self.d_sum = tf.summary.histogram("d", self.D)
        self.d__sum = tf.summary.histogram("d_", self.D_)
        self.G_sum = tf.summary.image("G", self.G)
        
        self.d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = self.D_logits,
                                                                                  labels = 0.9*tf.ones_like(self.D)))
        self.d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = self.D_logits_,
                                                                                  labels = tf.zeros_like(self.D_)))
        self.g_loss0 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = self.D_logits_, 
                                                                              labels = tf.ones_like(self.D_)))


        #Feature Matching
        self.features_from_g = tf.reduce_mean(self.fm_, reduction_indices=(0))
        self.features_from_i = tf.reduce_mean(self.fm, reduction_indices=(0))
        self.fm_g_loss1 =tf.multiply(tf.nn.l2_loss(self.features_from_g - self.features_from_i), 0.1)

        self.mean_image_from_g = tf.reduce_mean(self.G, reduction_indices=(0))
        self.mean_image_from_i = tf.reduce_mean(self.images, reduction_indices=(0))
        self.fm_g_loss2 = tf.multiply(tf.nn.l2_loss(self.mean_image_from_g - self.mean_image_from_i), 0.01)


        self.d_loss_real_sum = tf.summary.scalar("d_loss_real", self.d_loss_real)
        self.d_loss_fake_sum = tf.summary.scalar("d_loss_fake", self.d_loss_fake)

        self.d_loss = self.d_loss_real + self.d_loss_fake
        self.g_loss = self.g_loss0 + self.fm_g_loss1 + self.fm_g_loss2

        self.g_loss_sum = tf.summary.scalar("g_loss", self.g_loss)
        self.d_loss_sum = tf.summary.scalar("d_loss", self.d_loss)

        t_vars = tf.trainable_variables()

        self.d_vars = [var for var in t_vars if 'd_' in var.name]
        self.g_vars = [var for var in t_vars if 'g_' in var.name]

        self.saver = tf.train.Saver()
        
    def train(self, config):
    
#         if config.dataset == 'MidiNet_v1':
            # change the file path to your dataset
        print("Begin training...")
        data_X = np.load('/Users/sbnahata/Documents/University of Michigan/EECS 545/Project_local/data_x.npy')
        data_X = np.reshape(data_X, [128, 16, -1, 1])
        data_X = np.transpose(data_X,(2,1,0,3))
        
        prev_X = np.load('/Users/sbnahata/Documents/University of Michigan/EECS 545/Project_local/prev_x.npy')
        prev_X = np.reshape(prev_X, [128, 16, -1, 1])
        prev_X = np.transpose(prev_X,(2,1,0,3))
        
        data_y = np.load('/Users/sbnahata/Documents/University of Michigan/EECS 545/Project_local/data_y.npy')
        data_y = np.transpose(data_y,(1,0))

#             data_X, prev_X, data_y = shuffle(data_X,prev_X,data_y, random_state=0)
            
#             data_X = np.transpose(data_X,(0,2,3,1))
#             prev_X = np.transpose(prev_X,(0,2,3,1))
#             print (prev_X.shape)
        
        d_optim = tf.train.AdamOptimizer(config.learning_rate, beta1=config.beta1) \
                          .minimize(self.d_loss, var_list=self.d_vars)
        g_optim = tf.train.AdamOptimizer(config.learning_rate, beta1=config.beta1) \
                          .minimize(self.g_loss, var_list=self.g_vars)
        tf.global_variables_initializer().run()

        self.g_sum = tf.summary.merge([self.z_sum, self.d__sum, 
            self.G_sum, self.d_loss_fake_sum, self.g_loss_sum])
        self.d_sum = tf.summary.merge([self.z_sum, self.d_sum, self.d_loss_real_sum, self.d_loss_sum])
        self.writer = tf.summary.FileWriter("./logs", self.sess.graph)

        sample_z = np.random.normal(0, 1, size=(self.sample_size , self.z_dim))
        sample_files = data_X[0:self.sample_size]
        
        save_images(data_X[np.arange(len(data_X))[:5]]*1, [1, 5],
        './{}/Train.png'.format(config.sample_dir))
        
        
        sample_images = data_X[0:self.sample_size]
        counter = 0
        start_time = time.time()

#         if self.load(self.checkpoint_dir):
#             print(" [*] Load SUCCESS")
#         else:
#             print(" [!] Load failed...")
        

        sample_labels = sloppy_sample_labels()
        print("Starting epochs...")
        for epoch in xrange(config.epoch):
            
            batch_idxs = len(data_X) // config.batch_size
            
            

            for idx in xrange(0, batch_idxs):
                
                batch_images = data_X[idx*config.batch_size:(idx+1)*config.batch_size]
                prev_batch_images = prev_X[idx*config.batch_size:(idx+1)*config.batch_size]
                
                batch_labels = data_y[idx*config.batch_size:(idx+1)*config.batch_size]
                '''
                Note that the mu and sigma are set to (-1,1) in the experiment of the paper :
                "MidiNet: A Convolutional Generative Adversarial Network for Symbolic-domain Music Generation"
                However, the result are similar by using (0,1)
                '''
                batch_z = np.random.normal(0, 1, [config.batch_size, self.z_dim]) \
                            .astype(np.float32)

                
                # Update D network
                _, summary_str = self.sess.run([d_optim, self.d_sum],
                    feed_dict={ self.images: batch_images, self.z: batch_z ,self.y:batch_labels, self.prev_bar:prev_batch_images })
                self.writer.add_summary(summary_str, counter)

                # Update G network
                _, summary_str = self.sess.run([g_optim, self.g_sum],
                        feed_dict={ self.images: batch_images, self.z: batch_z ,self.y:batch_labels, self.prev_bar:prev_batch_images })
                self.writer.add_summary(summary_str, counter)

                # Run g_optim twice to make sure that d_loss does not go to zero (different from paper)
                # We've tried to run more d_optim and g_optim, while getting a better result by running g_optim twice in this MidiNet version.
                _, summary_str = self.sess.run([g_optim, self.g_sum],
                        feed_dict={ self.images: batch_images, self.z: batch_z ,self.y:batch_labels, self.prev_bar:prev_batch_images })
                self.writer.add_summary(summary_str, counter)
                    
                errD_fake = self.d_loss_fake.eval({self.z: batch_z, self.y:batch_labels, self.prev_bar:prev_batch_images })
                errD_real = self.d_loss_real.eval({self.images: batch_images, self.y:batch_labels })
                errG = self.g_loss.eval({self.images: batch_images, self.z: batch_z, self.y:batch_labels, self.prev_bar:prev_batch_images })
                
                
                

                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, d_loss: %.8f, g_loss: %.8f" \
                    % (epoch, idx, batch_idxs,
                        time.time() - start_time, errD_fake+errD_real, errG))

                if np.mod(counter, 100) == 1:
                    
                    samples, d_loss, g_loss = self.sess.run(
                        [self.sampler, self.d_loss, self.g_loss],
                        feed_dict={self.z: sample_z, self.images: sample_images, self.y:sample_labels, self.prev_bar:prev_batch_images }
                    )
                    #samples = (samples+1.)/2.
                    save_images(samples[:5,:], [1, 5],
                                './{}/train_{:02d}_{:04d}.png'.format(config.sample_dir, epoch, idx))
                    print("[Sample] d_loss: %.8f, g_loss: %.8f" % (d_loss, g_loss))

                    np.save('./{}/train_{:02d}_{:04d}'.format(config.gen_dir,  epoch, idx), samples)

                if np.mod(counter, len(data_X)//config.batch_size) == 0:
                    print("Saving Checkpoint")
                    self.save(config.checkpoint_dir, counter)
            print("Epoch: [%2d] time: %4.4f, d_loss: %.8f" \
            % (epoch, 
                time.time() - start_time, (errD_fake+errD_real)/batch_idxs))
    
    def generator(self, z, y, prev_x=None):
        with tf.variable_scope("generator") as scope:
            print("GENERATOR")
            h0_prev = lrelu(self.g_prev_bn0(conv2d(prev_x, 16, k_h=1, k_w=128,d_h=1, d_w=2, name='g_h0_prev_conv')))
            h1_prev = lrelu(self.g_prev_bn1(conv2d(h0_prev, 16, k_h=2, k_w=1, name='g_h1_prev_conv')))
            h2_prev = lrelu(self.g_prev_bn2(conv2d(h1_prev, 16, k_h=2, k_w=1, name='g_h2_prev_conv')))
            h3_prev = lrelu(self.g_prev_bn3(conv2d(h2_prev, 16, k_h=2, k_w=1, name='g_h3_prev_conv')))

            yb = tf.reshape(y, [self.batch_size, 1, 1, self.y_dim])
            z = tf.concat([z, y], 1)

            h0 = tf.nn.relu(self.g_bn0(linear(z, 1024, 'g_h0_lin')))
            h0 = tf.concat([h0, y], 1)

            h1 = tf.nn.relu(self.g_bn1(linear(h0, self.gf_dim*2*2*1, 'g_h1_lin')))

            h1 = tf.reshape(h1, [self.batch_size, 2, 1, self.gf_dim * 2])
            h1 = conv_cond_concat(h1, yb)
            h1 = conv_prev_concat(h1, h3_prev)

            h2 = tf.nn.relu(self.g_bn2(deconv2d(h1, [self.batch_size, 4, 1, self.gf_dim * 2],
                                                k_h=2, k_w=1,d_h=2, d_w=2 ,name='g_h2')))
            h2 = conv_cond_concat(h2, yb)
            h2 = conv_prev_concat(h2, h2_prev)

            h3 = tf.nn.relu(self.g_bn3(deconv2d(h2, [self.batch_size, 8, 1, self.gf_dim * 2],k_h=2, k_w=1,d_h=2, d_w=2 ,name='g_h3')))
            h3 = conv_cond_concat(h3, yb)
            h3 = conv_prev_concat(h3, h1_prev)

            h4 = tf.nn.relu(self.g_bn4(deconv2d(h3, [self.batch_size, 16, 1, self.gf_dim * 2],k_h=2, k_w=1,d_h=2, d_w=2 ,name='g_h4')))
            h4 = conv_cond_concat(h4, yb)
            h4 = conv_prev_concat(h4, h0_prev)

            return tf.nn.sigmoid(deconv2d(h4, [self.batch_size, 16, 128, self.c_dim],k_h=1, k_w=128,d_h=1, d_w=2, name='g_h5'))

    def discriminator(self, x, y=None, reuse=False):
        with tf.variable_scope("discriminator") as scope:
            if reuse:
                scope.reuse_variables()
            df_dim = 64
            dfc_dim = 1024

            print("DISCRIMINATOR")
            yb = tf.reshape(y, [self.batch_size, 1, 1, self.y_dim])

            x = conv_cond_concat(x, yb)

            h0 = lrelu(conv2d(x, self.c_dim + self.y_dim,k_h=2, k_w=128, name='d_h0_conv'))
            fm = h0
            h0 = conv_cond_concat(h0, yb)

            h1 = lrelu(self.d_bn1(conv2d(h0, self.df_dim + self.y_dim,k_h=4, k_w=1, name='d_h1_conv')))
            h1 = tf.reshape(h1, [self.batch_size, -1])    
            h1 = tf.concat([h1, y],1)

            h2 = lrelu(self.d_bn2(linear(h1, dfc_dim, 'd_h2_lin')))
            h2 = tf.concat([h2, y],1)

            h3 = linear(h2, 1, 'd_h3_lin')

            return tf.nn.sigmoid(h3), h3, fm
    
    def sampler(self, z, y=None, prev_x=None):
        with tf.variable_scope("generator") as scope:
            scope.reuse_variables()
            print("SAMPLER")
            tf.get_variable_scope().reuse_variables()
            h0_prev = lrelu(self.g_prev_bn0(conv2d(prev_x, 16, k_h=1, k_w=128, d_h=1, d_w=2,name='g_h0_prev_conv')))
            print(h0_prev)
            h1_prev = lrelu(self.g_prev_bn1(conv2d(h0_prev, 16, k_h=2, k_w=1, name='g_h1_prev_conv')))
            h2_prev = lrelu(self.g_prev_bn2(conv2d(h1_prev, 16, k_h=2, k_w=1, name='g_h2_prev_conv')))
            h3_prev = lrelu(self.g_prev_bn3(conv2d(h2_prev, 16, k_h=2, k_w=1, name='g_h3_prev_conv')))


            yb = tf.reshape(y, [self.batch_size, 1, 1, self.y_dim])
            z = tf.concat([z, y],1)

            h0 = tf.nn.relu(self.g_bn0(linear(z, 1024, 'g_h0_lin')))
            h0 = tf.concat([h0, y],1)

            h1 = tf.nn.relu(self.g_bn1(linear(h0, self.gf_dim*2*2*1, 'g_h1_lin')))

            h1 = tf.reshape(h1, [self.batch_size, 2, 1, self.gf_dim * 2])
            h1 = conv_cond_concat(h1, yb)
            h1 = conv_prev_concat(h1, h3_prev)

            h2 = tf.nn.relu(self.g_bn2(deconv2d(h1, [self.batch_size, 4, 1, self.gf_dim * 2],k_h=2, k_w=1,d_h=2, d_w=2 ,name='g_h2')))
            h2 = conv_cond_concat(h2, yb)
            h2 = conv_prev_concat(h2, h2_prev)

            h3 = tf.nn.relu(self.g_bn3(deconv2d(h2, [self.batch_size, 8, 1, self.gf_dim * 2],k_h=2, k_w=1,d_h=2, d_w=2 ,name='g_h3')))
            h3 = conv_cond_concat(h3, yb)
            h3 = conv_prev_concat(h3, h1_prev)

            h4 = tf.nn.relu(self.g_bn4(deconv2d(h3, [self.batch_size, 16, 1, self.gf_dim * 2],k_h=2, k_w=1,d_h=2, d_w=2 ,name='g_h4')))
            h4 = conv_cond_concat(h4, yb)
            h4 = conv_prev_concat(h4, h0_prev)

            return tf.nn.sigmoid(deconv2d(h4, [self.batch_size, 16, 128, self.c_dim],k_h=1, k_w=128,d_h=1, d_w=2, name='g_h5'))
        
    def save(self, checkpoint_dir, step):
        model_name = "MidiNet.model"
        model_dir = "%s_%s_%s" % ("MIDIGAN", self.batch_size, self.output_w)
        checkpoint_dir = os.path.join(checkpoint_dir, model_dir)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,
                        os.path.join(checkpoint_dir, model_name),
                        global_step=step)

In [3]:
flags = tf.app.flags
flags.DEFINE_integer("epoch", 2, "Epoch to train [20]")
flags.DEFINE_float("learning_rate", 0.00005, "Learning rate of for adam [0.0002]")
flags.DEFINE_float("beta1", 0.5, "Momentum term of adam [0.5]")
flags.DEFINE_integer("batch_size", 72, "The size of batch [72]")
flags.DEFINE_integer("output_w", 16, "The size of the output segs to produce [16]")
flags.DEFINE_integer("output_h", 128, "The size of the output note to produce [128]")
flags.DEFINE_integer("c_dim", 1, "Number of Midi track. [1]")
flags.DEFINE_string("checkpoint_dir", "checkpoint", "Directory name to save the checkpoints [checkpoint]")
flags.DEFINE_string("sample_dir", "samples", "Directory name to save the image samples [samples]")
flags.DEFINE_string("dataset", "MidiNet_v1", "The name of dataset ")
flags.DEFINE_boolean("is_train", True, "True for training, False for testing [False]")
flags.DEFINE_boolean("is_crop", False, "True for training, False for testing [False]")
flags.DEFINE_boolean("generation_test", False, "True for generation_test, False for nothing [False]")
flags.DEFINE_string("gen_dir", "gen", "Directory name to save the generate samples [samples]")
FLAGS = flags.FLAGS

In [4]:
def main(_):
    pp.pprint(flags.FLAGS.__flags)

    if not os.path.exists(FLAGS.checkpoint_dir):
        os.makedirs(FLAGS.checkpoint_dir)
    if not os.path.exists(FLAGS.sample_dir):
        os.makedirs(FLAGS.sample_dir)
    if not os.path.exists(FLAGS.gen_dir):
        os.makedirs(FLAGS.gen_dir)

    with tf.Session() as sess:
        if FLAGS.dataset == 'MidiNet_v1':
            model = MidiNet(sess,  batch_size=FLAGS.batch_size,y_dim=13, output_w=FLAGS.output_w, 
                            output_h=FLAGS.output_h, c_dim=FLAGS.c_dim)
        
        if FLAGS.is_train:
            model.train(FLAGS)
            
        k = generation_test (sess, model, FLAGS, 0)
        print(k)
        k = np.transpose(k,(2,1,0,3))
        k = np.reshape(k,[128, -1])
        print(k)
        pm = piano_roll_to_pretty_midi(k)
        pm.write('/Users/sbnahata/Documents/University of Michigan/EECS 545/Project_local/FINALLY.mid')
#         else:
#             model.load(FLAGS.checkpoint_dir)

        

if __name__ == '__main__':
    tf.app.run()

{'batch_size': 72,
 'beta1': 0.5,
 'c_dim': 1,
 'checkpoint_dir': 'checkpoint',
 'dataset': 'MidiNet_v1',
 'epoch': 2,
 'gen_dir': 'gen',
 'generation_test': False,
 'is_crop': False,
 'is_train': True,
 'learning_rate': 5e-05,
 'output_h': 128,
 'output_w': 16,
 'sample_dir': 'samples'}
GENERATOR
DISCRIMINATOR
SAMPLER
Tensor("generator_1/Maximum:0", shape=(72, 16, 1, 16), dtype=float32)
DISCRIMINATOR
Begin training...
Starting epochs...
Epoch: [ 0] [   0/   3] time: 0.9858, d_loss: 1.42000544, g_loss: 299.00064087
[Sample] d_loss: 1.41767430, g_loss: 299.00888062
Epoch: [ 0] [   1/   3] time: 1.6202, d_loss: 1.39558387, g_loss: 258.14575195
Epoch: [ 0] [   2/   3] time: 2.1536, d_loss: 1.38155150, g_loss: 268.47909546
Saving Checkpoint
Epoch: [ 0] time: 2.6542, d_loss: 0.46051717
Epoch: [ 1] [   0/   3] time: 3.3018, d_loss: 1.36324549, g_loss: 298.83688354
Epoch: [ 1] [   1/   3] time: 3.8264, d_loss: 1.34504175, g_loss: 257.93237305
Epoch: [ 1] [   2/   3] time: 4.4720, d_loss: 1.33

SystemExit: 

/Users/sbnahata/anaconda3/envs/tensorflow/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
